In [149]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaspilab/salary_train.csv
/kaggle/input/kaspilab/salary_example_submition.csv
/kaggle/input/kaspilab/salary_predict.csv


In [142]:
import pandas as pd
import numpy as np

In [143]:
data = pd.read_csv('salary_train.csv')

In [144]:
data_predict = pd.read_csv('salary_predict.csv')

In [145]:
data

,Id,algebra,programming,data science,robotics,economics,job,salary
0,0,87,62,86,61,90,junior developer,140000
1,1,76,84,76,80,79,data scientist,780000
2,2,56,55,99,82,98,developer,210000
3,3,99,66,65,84,58,economist,420000
4,4,73,87,56,84,73,data scientist,760000
...,...,...,...,...,...,...,...,...
8995,8995,58,85,68,62,97,senior developer,590000
8996,8996,92,58,99,77,81,robotics engineer,1050000
8997,8997,92,54,81,63,74,developer,300000
8998,8998,98,90,51,96,56,developer,420000


In [146]:
data.drop(['Id'], axis=1, inplace=True)

In [147]:
data_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            1000 non-null   int64 
 1   algebra       1000 non-null   int64 
 2   programming   1000 non-null   int64 
 3   data science  1000 non-null   int64 
 4   robotics      1000 non-null   int64 
 5   economics     1000 non-null   int64 
 6   job           1000 non-null   object
 7   salary        1000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 62.6+ KB


In [148]:
data_predict.drop(['Id'], axis=1, inplace=True)

In [149]:
one_hot_encoded_data = pd.get_dummies(data['job'],prefix='job')
data = data.join(one_hot_encoded_data)

In [150]:
data.drop(['job'], axis = 1, inplace = True)

In [151]:
one_hot_encoded_predict = pd.get_dummies(data_predict['job'],prefix='job')
data_predict = data_predict.join(one_hot_encoded_predict)

In [152]:
data_predict.drop(['job'], axis = 1, inplace = True)

In [153]:
data_submission = pd.read_csv('salary_example_submition.csv')

In [161]:
for i in range(len(data_submission)) : 
   if data_submission.loc[i, "salary"]>1000000:
        data_submission.loc[i, "salary"]=1000000
    
    
print(data_submission['salary'].head(100))

        
     

0     890875
1     175973
2     748288
3     510000
4     410965
       ...  
95    419216
96    139530
97    897167
98    122682
99    290894
Name: salary, Length: 100, dtype: int64


In [162]:
data_submission

,Id,salary
0,9000,890875
1,9001,175973
2,9002,748288
3,9003,510000
4,9004,410965
...,...,...
995,9995,375806
996,9996,418279
997,9997,411755
998,9998,130651


In [131]:
from sklearn.model_selection import train_test_split

In [132]:
y = data['salary']
df = data.drop(['salary'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [133]:

from sklearn import datasets, linear_model
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
print("Score:", model.score(X_test, y_test))

Score: 0.948280737078265


In [134]:
x_pred = data_predict.drop(['salary'], axis=1)

In [139]:
predictions = model.predict(x_pred)
data_submission['salary'] = predictions
data_submission['salary'] = data_submission["salary"].astype(int)




In [141]:
data_submission
    

,Id,salary
0,9000,841659
1,9001,563874
2,9002,340613
3,9003,1003786
4,9004,177575
...,...,...
995,9995,244483
996,9996,373220
997,9997,300714
998,9998,120670


In [163]:
data_submission.to_csv('submission_linear3.csv', index=False)

In [166]:
from catboost import CatBoostRegressor
RANDOM_SEED = 42 
model = CatBoostRegressor(iterations = 5000, random_seed = RANDOM_SEED, eval_metric='MAPE', 
                          custom_metric=['R2', 'MAE'], silent=True)

ModuleNotFoundError: No module named 'catboost'

In [197]:
model.fit(X_train, y_train,   eval_set=(X_test, y_test),
     verbose_eval=0,
     use_best_model=True,
     #plot=True
     )

In [198]:
print("Score:", model.score(X_test, y_test))

Score: 0.968393666940298


In [164]:
predictions_cat = model.predict(x_pred)
data_submission['salary'] = predictions_cat